# DREAM diagnostics

In [ ]:
%matplotlib widget
import pandas as pd
import scipp as sc
import plopp as pp
import matplotlib.pyplot as plt
from ess.dream import diagnostics as diags
from ess import dream

In [ ]:
filename = 'data_dream_HF_mil_closed_alldets_1e9.csv.zip'
df = pd.read_table(filename)
ds = sc.compat.from_pandas(df)
del ds.coords['row']  # we have no use for this row index
for key in list(ds):
    name, *remainder = key.split(' ')
    ds[name] = ds.pop(key)
    ds[name].unit = remainder[0][1:-1] if remainder else None
ds['lambda'].unit = 'angstrom'
table = sc.DataArray(sc.ones(sizes=ds.sizes, unit='counts'))
for name in ds:
    table.coords[name] = ds[name].data

mapping = {'sumo0': 0, 'sumo1': 1, 'sumo2': 2, 'sumo3': 3, 'mantle': 4, 'high_res': 5}
da = table.group('det')
dg = sc.DataGroup({key: da['det', val] for key, val in mapping.items()})
dg = sc.DataGroup({key: diags.to_logical_dims(da) for key, da in dg.items()})
for da in dg.values():
    da.coords['x'] = da.bins.coords.pop('x_pos').bins.mean()
    da.coords['y'] = da.bins.coords.pop('y_pos').bins.mean()
    da.coords['z'] = da.bins.coords.pop('z_pos').bins.mean()
dg

In [ ]:
fig, ax = plt.subplots(4, len(dg), figsize=(len(dg) * 3, 4 * 3))
for i, (name, da) in enumerate(dg.items()):
    da.bins.concat(list(set(da.dims) - {'module'})).hist().plot(ax=ax[0, i], title=name)
    diags.module_vs_segment(da).hist().plot(ax=ax[1, i])
    da.bins.concat(list(set(da.dims) - {'strip'})).hist().plot(ax=ax[2, i])
    diags.wire_vs_strip(da).hist().plot(ax=ax[3, i])
fig.subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
dream.instrument_view(dg, dim='tof', bins=10)